# Abgeordnetenwatch Emails

### Augment Abgeordnetenwatch data to include Abgeordnete (representives) email

Processes the Bundestag-Abgeordnete.csv from https://www.abgeordnetenwatch.de/ and adds email addresses for each representative as accurately as possible

## Method

Creates email addresses based on first_name, last_name

This process tries to use the same convention as the Bundestag uses when forming email addresses.

All conventions are based on observation and may not be entirely correct.

### Observed conventions in email formatting:

- Hyphenated "-" last names such as Aschenberg-Dugnus

  Preserved in the email address
- Hyphenated "-" first names such as Leif-Erik

  Preserved in the email address
- All accented characters "äéöüß" are converted

  See accent_aliases dictionary below
- If there are multiple first names, there is no consistency to whether they are included or not and how. Most of the time the second name is just dropped, but not always.

  For these occurances, the known_emails lookup is used to explicity set the email address

- If a last name contains "von", there is no consistency to whether it is included or not and how.

  For these occurances, the known_emails lookup is used to explicity set the email address
- If a last name starts with "de " as in "de Vries" the "de" is preserved and the space removed

## Usage

_Free to use, duplicate, whatever._

Unzip contents of data zipfile to project directory then:

```
pip install -r requirements.txt
jupyter notebook
```


In [14]:
import pandas as pd
reps = pd.read_csv('./Daten-abgeordnetenwatch/Bundestag-Abgeordnete.csv', sep=';')
reps.columns = reps.iloc[0]
reps = reps[1:]
constituencies = pd.read_csv('./Daten-abgeordnetenwatch/wahlkreise_de_bundestag.csv', sep=';')
constituencies.columns = constituencies.iloc[0]
constituencies = constituencies[1:]


In [16]:
# Setup for email generation

import re
# A helper regular expression to match on any unexpected characters like spaces, dashes etc etc
non_alphanum_regex = re.compile(r'[^a-z0-9äöü]+', re.IGNORECASE)

accent_aliases = {
    'Ä': 'Ae',
    'Ö': 'Oe',
    'Ü': 'Ue',
    'ä': 'ae',
    'ö': 'oe',
    'ü': 'ue',
    'é': 'e',
    'è': 'e',
    'ß': 'ss',
}

accent_translate_map = {ord(key):val for key, val in accent_aliases.items()}

# If emails don't follow any common rules, they can be set here to ensure accuracy
# Lookup key should be of the form f'{first_name} {last_name}' where first_name and last_name are the values taken from the corresponding columns in the dataset
known_emails = {
    'Jan R. Nolte': 'jan.nolte@bundestag.de',
    'Lorenz Gösta Beutin': 'lorenz.beutin@bundestag.de',
    'Ernst Dieter Rossmann': 'ernst-dieter.rossmann@bundestag.de',
    'Michael von Abercron': 'michael.vonabercron@bundestag.de',
    'Konstantin von Notz': 'konstantin.notz@bundestag.de',
    'Amira Mohamed Ali': 'amira.mohamedali@bundestag.de',
    'Armin Paul Hampel': 'armin.paulus.hampel@bundestag.de',
    'Ottmar von Holtz': 'ottmar.vonholtz@bundestag.de',
    'Hans-Georg von der Marwitz': 'hans-georg.vondermarwitz@bundestag.de',
    'Beatrix von Storch': 'beatrix.vonstorch@bundestag.de',
    'Reinhard Arnold Houben': 'reinhard.houben@bundestag.de',
    'Matthias W. Birkwald': 'matthias-w.birkwald@bundestag.de',
    'Alexander Graf Lambsdorff': 'alexander.graflambsdorff@bundestag.de',
    'Alexander S. Neu': 'alexander.neu@bundestag.de',
    'Olaf in der Beek': 'olaf.inderbeek@Bundestag.de',
    'Berengar Elsner von Gronow': 'berengar.elsnervongronow@bundestag.de',
    'Jan R. Nolte': 'jan.nolte@bundestag.de',
    'Hermann Otto Solms': 'hermann.solms@bundestag.de',
    'Tobias Matthias Peterka': 'tobias.peterka@bundestag.de',
    'Christian von Stetten': 'christian.stetten@bundestag.de',
    'Axel Eduard Fischer': 'axel.fischer@bundestag.de',
    'Karl A. Lamers': 'karl-a.lamers@bundestag.de',
    'Matern von Marschall': 'matern.vonmarschall@bundestag.de',
    'Helin Evrim Sommer': 'helin-evrim.sommer@bundestag.de',
    'Wilhelm von Gottberg': 'wilhelm.vongottberg@bundestag.de',
}



In [28]:
# Email generation
emails = []

for rep in reps.itertuples():
    # Uncomment to see "flagged" records (ie names that might not work when used to build an email address)
    # if non_alphanum_regex.search(rep.first_name + rep.last_name):
    #     print(f'"{rep.first_name}" "{rep.last_name}"')

    try:
        # Is there an explicit email address set for this name?
        email = known_emails[f'{rep.first_name} {rep.last_name}']
    except KeyError:
        # There is not, so we generate it to the best of our knowledge
        first_name = rep.first_name.lower().strip()
        last_name = rep.last_name.lower().strip()

        if last_name.startswith('de '):
            # 'de Vries', 'De Riddler' etc
            last_name = last_name.replace('de ', 'de')

        email = f'{first_name}.{last_name}@bundestag.de'.translate(accent_translate_map)
    emails.append(email)

print(len(emails), 'emails generated')


709 emails generated


In [29]:
# Append emails row to the dataframe
reps['email'] = emails



In [34]:
# The datasource now has emails!
reps.head()



,id,politician_id,parliament_period,last_name,first_name,sex,party_politician,fraction,constituency_name,constituency_number,constituency_result,occupation,education,year_of_birth,profile_url,email
1,41857,130900,Bundestag,Nicolaisen,Petra,f,CDU,CDU/CSU,1 - Flensburg - Schleswig (Bundestag),1,40.00 %,MdB,Industriekauffrau,1965,https://www.abgeordnetenwatch.de/profile/petra...,petra.nicolaisen@bundestag.de
2,43685,139019,Bundestag,Damerow,Astrid,f,CDU,CDU/CSU,2 - Nordfriesland - Dithmarschen Nord (Bundestag),2,45.10 %,MdB,Bankkauffrau,1958,https://www.abgeordnetenwatch.de/profile/astri...,astrid.damerow@bundestag.de
3,44536,79299,Bundestag,Helfrich,Mark,m,CDU,CDU/CSU,3 - Steinburg - Dithmarschen Süd (Bundestag),3,41.90 %,MdB,Diplom Kaufmann (FH),1978,https://www.abgeordnetenwatch.de/profile/mark-...,mark.helfrich@bundestag.de
4,41227,122157,Bundestag,Kubicki,Wolfgang,m,FDP,FDP,3 - Steinburg - Dithmarschen Süd (Bundestag),3,11.00 %,MdB,Rechtsanwalt,1952,https://www.abgeordnetenwatch.de/profile/wolfg...,wolfgang.kubicki@bundestag.de
5,41132,120104,Bundestag,Nestle,Ingrid,f,DIE GRÜNEN,DIE GRÜNEN,3 - Steinburg - Dithmarschen Süd (Bundestag),3,6.70 %,MdB,Diplom-Wirtschaftsingenieurin,1977,https://www.abgeordnetenwatch.de/profile/ingri...,ingrid.nestle@bundestag.de


In [35]:
# Save it, using a comma as a separator this time!
reps.to_csv('./Daten-abgeordnetenwatch/Bundestag-Abgeordnete-emails.csv', sep=',')